In [1]:
import os
import random
import wandb
import torch
import numpy as np
import matplotlib.pyplot as plt
from beir.datasets.data_loader import GenericDataLoader

from matryoshka import Matryoshka

/Users/mikolajboronski/miniconda3/envs/d/lib/python3.11/site-packages/beir/datasets/data_loader.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
data_path = "data/nfcorpus"
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="train")

length = None
corpus = {k: v for k, v in list(corpus.items())[:length]}
queries = {k: v for k, v in list(queries.items())[:length]}
qrels = {k: v for k, v in list(qrels.items())[:length]}

100%|██████████| 3633/3633 [00:00<00:00, 225957.66it/s]


In [3]:
base_model = Matryoshka(matryoshka_dim=384, adaptor=False)
model = Matryoshka(matryoshka_dim=384, adaptor=True)
tokenizer = model.tokenizer

sentences = ["sentence"]
inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)

/Users/mikolajboronski/miniconda3/envs/d/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Users/mikolajboronski/miniconda3/envs/d/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# sanity check on shape
model(pooling=False, **inputs).shape

torch.Size([1, 3, 384])

In [5]:
# sanity check on learble parameters
for i in model.named_parameters():
    if i[1][1].requires_grad:
        print(i[0])

adaptor.down_project.weight
adaptor.down_project.bias
adaptor.up_project.weight
adaptor.up_project.bias


##### Trying to reproduce original embeddings with adaptor on

In [6]:
cs = [c["text"] for c in corpus.values()]
qs = list(queries.values())

In [7]:
epochs = 100
batch_size = 32
running_loss_step = 10
learning_rate = 1e-5

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
similarity_loss = torch.nn.CosineEmbeddingLoss()

data = qs + cs
random.shuffle(data)

wandb.init(
    project="matryoshka",  
    name="your-run-name",        
    config={                      
        "learning_rate": learning_rate,
        "epochs": epochs,
        "batch_size": batch_size,
        "loss": "MSE",
        "model": model.name,
        "model_card": model.model_card_data,
        "loss_resolution": running_loss_step,
    }
)

ls = []
for i in range(100):
    epoch_loss = []
    running_loss = []
    for j in range(0, len(data), batch_size):
        if j + batch_size > len(data):
            break
        # c = cs[j : j + 32]
        # q = qs[j : j + 32]
        q = data[j : j + batch_size]
        inputs = tokenizer(q, return_tensors="pt", padding=True, truncation=True)
        outputs = model(pooling=False, **inputs)
        target_outputs = base_model(pooling=False, **inputs)
        loss = torch.nn.MSELoss()(outputs, target_outputs)
        # similarity_loss = similarity_loss(outputs, target_outputs, torch.ones(len(q)))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        ls.append(loss.item())
        epoch_loss.append(loss.item())
        running_loss.append(loss.item())
        if len(running_loss) % 10 == 0:
            print("Batch:", j, "loss:", np.mean(running_loss))
            wandb.log({"batch": j, "loss": np.mean(running_loss)})
            running_loss = []

    wandb.log({"epoch": i, "loss": np.mean(ls)})
    print("Epoch:", i, "loss:", np.mean(ls))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key f

KeyboardInterrupt: 